In [1]:
import numpy as np 
import pandas as pd 
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

from tqdm import tqdm
from google.colab import drive 
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/jane_street")

Mounted at /content/drive/


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from maf import MAF, RealNVP
from sklearn.metrics import  precision_score, recall_score, f1_score
from IPython import embed
from sklearn.preprocessing import MinMaxScaler

LATENT_DIM = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1. PurgedGroupTimeSeriesSplit
From [here](https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv), thx for sharing

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

## 2. Preprocessing

In [4]:
# train = pd.read_csv('train.csv')
# train = train.query('date > 85').reset_index(drop = True) 
# train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
# train.fillna(train.mean(),inplace=True)
# train = train.query('weight > 0').reset_index(drop = True)

# train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
# features = [c for c in train.columns if 'feature' in c]

# resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

# X = train[features].values
# y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget

# f_mean = np.mean(train[features[1:]].values,axis=0)

In [4]:
class CustomDataset:
    def __init__(self, dataset, target):
        self.dataset = dataset
        self.target = target

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        return {
            'x': torch.tensor(self.dataset[item, :], dtype=torch.float),
            'y': torch.tensor(self.target[item, :], dtype=torch.long)
        }

def load_data(PATH):
    dt = pd.read_csv(PATH)
    dt = pd.DataFrame(dt)
    dt['action'] = (dt['resp'] > 0).astype('int')
    dt.drop(columns=['resp', 'date', 'ts_id'], inplace=True)
    
    return dt

data = load_data('train.csv')
data.fillna(-1, inplace=True)
target_column = 'action'
feature_columns = data.columns[~data.columns.isin([target_column])]

random_seed = 1
learning_rate = 0.1
num_epochs = 1
batch_size = 2048
num_features = len(feature_columns)
num_hidden_1 = 128
num_hidden_2 = 64
num_classes = 2

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

train, validation = data[:int(len(data) * 0.75)], data[int(len(data) * 0.75):]

train_data, train_target = train[feature_columns], train[[target_column]]
validation_data, validation_target = validation[feature_columns], validation[[target_column]]
train_dataset = CustomDataset(dataset=train_data.values, target=train_target.values)
validation_dataset = CustomDataset(dataset=validation_data.values, target=validation_target.values)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

IndexError: ignored

## 3. AutoEncoder
THX for sharing [this great work](https://www.kaggle.com/snippsy/bottleneck-encoder-mlp-keras-tuner)

In [29]:
def softclip(tensor, min):
    """ Clips the tensor values at the minimum value min in a softway. Taken from Handful of Trials """
    result_tensor = min + F.softplus(tensor - min)
    return result_tensor

class CNN_sigmaVAE(nn.Module):

    def __init__(self,latent_dim=LATENT_DIM, window_size=20, use_probabilistic_decoder=False):
        super(CNN_sigmaVAE, self).__init__()
        
        self.window_size=window_size
        self.latent_dim = latent_dim
        self.prob_decoder = use_probabilistic_decoder
        
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(8)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(16)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=4, kernel_size=5, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(4)
        
        
        self.fc41 = nn.Linear(4*123, self.latent_dim)
        self.fc42 = nn.Linear(4*123, self.latent_dim)

        self.defc1 = nn.Linear(self.latent_dim, 4*123)
        
        self.deconv1 = nn.ConvTranspose1d(in_channels=4, out_channels=16, kernel_size=5, stride=1, padding=0, output_padding=0)
        self.debn1 = nn.BatchNorm1d(16)
        self.deconv2 = nn.ConvTranspose1d(in_channels=16, out_channels=8, kernel_size=5, stride=1, padding=0, output_padding=0)
        self.debn2 = nn.BatchNorm1d(8)
        self.deconv3 = nn.ConvTranspose1d(in_channels=8, out_channels=1, kernel_size=5, stride=1, padding=0, output_padding=0)

        self.log_sigma = 0
        self.log_sigma = torch.nn.Parameter(torch.full((1,), 0.0)[0], requires_grad=True)
        
        
        self.decoder_fc41 = nn.Linear(self.window_size, self.window_size)
        self.decoder_fc42 = nn.Linear(self.window_size, self.window_size)
        
        self.decoder_fc43 = nn.Linear(self.window_size, self.window_size)
        self.decoder_fc44 = nn.Linear(self.window_size, self.window_size)

        self.flow = MAF(n_blocks=1, input_size=2, cond_label_size=latent_dim, hidden_size=50, n_hidden=1)
        
    def encoder(self, x):
        concat_input = x #torch.cat([x, c], 1)
        h = self.bn1(F.relu(self.conv1(concat_input)))
        h = self.bn2(F.relu(self.conv2(h)))
        h = self.bn3(F.relu(self.conv3(h)))
        
        self.saved_dim = [h.size(1), h.size(2)]
        
        h = h.view(h.size(0), h.size(1) * h.size(2))
        # from IPython import embed
        # embed()
        return self.fc41(h), self.fc42(h)
    
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add(mu) # return z sample
    
    def decoder(self, z):
        concat_input = z #torch.cat([z, c], 1)
        concat_input = self.defc1(concat_input)
        concat_input = concat_input.view(concat_input.size(0), self.saved_dim[0], self.saved_dim[1])

        h = self.debn1(F.relu(self.deconv1(concat_input)))
        h = self.debn2(F.relu(self.deconv2(h)))     
        out = torch.sigmoid(self.deconv3(h))
        
        if self.prob_decoder:
            rec_mu = self.decoder_fc43(out).tanh()
            rec_sigma = self.decoder_fc44(out).tanh()
            return out, rec_mu, rec_sigma
        
        else:
            return out, 0, 0
    
    def forward(self, x):

        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        output, rec_mu, rec_sigma = self.decoder(z)

        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        return output, rec_mu, rec_sigma, kl_div


    def gaussian_nll(self, mu, log_sigma, x):
        return 0.5 * torch.pow((x - mu) / log_sigma.exp(), 2) + log_sigma + 0.5 * np.log(2 * np.pi)

    
    def reconstruction_loss(self, x_hat, x):

        log_sigma = self.log_sigma
        log_sigma = softclip(log_sigma, -6)
        
        rec_comps = self.gaussian_nll(x_hat, log_sigma, x)
        rec = rec_comps.sum()

        return rec_comps, rec

    
    def loss_function(self, recon_x, x, rec_mu, rec_sigma, kl):
        
        rec_comps, rec = self.reconstruction_loss(recon_x, x)
        #kl = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        rec_mu_sigma_loss = 0
        if self.prob_decoder:
            rec_mu_sigma_loss = self.gaussian_nll(rec_mu, rec_sigma, x).sum()
        
        return rec_comps, rec, rec_mu_sigma_loss, kl


def train_flow_model(model, num_epochs, learning_rate, dataloader):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    epochs=num_epochs
    tq = tqdm(range(epochs))

    torch.manual_seed(random_seed)
    for epoch in range(num_epochs):
        model.train()
        for iteration, batch in enumerate(dataloader):
            inputs = batch['x'].to(device)
            labels = batch['y'].to(device)
            labels = torch.squeeze(labels)

            inputs = inputs.unsqueeze(1)

            optimizer.zero_grad()

            mu, _ = model.encoder(inputs)
            labels = labels.unsqueeze(1).repeat(1, 2).float()
            zk, loss = model.flow.log_prob(x=labels, y=mu)

            # VAE Training
            # outputs, rec_mu, rec_sigma, kl = model(inputs)
            # _, rec, _, kl = model.loss_function(outputs, inputs, rec_mu, rec_sigma, kl)

            loss = -loss.mean()

            if(np.isnan(loss.item())):
                print("Noped out at", epoch, j, kl, rec_comps)

            loss.backward()
            optimizer.step()

        # tq.set_postfix(loss=loss.item())
        # print(epoch, 'total :' + str(loss.item()) + ' rec : ' + str(rec.item()) + ' kl : ' + str(kl.sum().item()) + ' sigma: ' + str(model.log_sigma.item()))
        print(epoch, 'total :' + str(loss.item()))

        torch.save(model, 'bitch_street_VAEflow.pth')
        torch.save(model.state_dict(), 'bitch_street_VAEflow_state_dict.pth')

        #break
    return model

def test_flow_model(model, dataloader):

    torch.manual_seed(random_seed)
    model.eval()
    for iteration, batch in enumerate(dataloader):
      inputs = batch['x'].to(device)
      labels = batch['y'].to(device)
      labels = torch.squeeze(labels)
      # inputs = torch.from_numpy(validation_data.to_numpy()).float().to(device)
      # labels = torch.from_numpy(validation_target.to_numpy()).float().to(device)

      labels = torch.squeeze(labels)
      inputs = inputs.unsqueeze(1)

      mu, _ = model.encoder(inputs)
      zero_test = torch.zeros([batch_size, 2]).to(device)
      ones_test = torch.ones_like(zero_test).to(device)

      _, zero_log_prob = model.flow.log_prob(x=zero_test, y=mu)
      _, one_log_prob = model.flow.log_prob(x=ones_test, y=mu)

      z = torch.zeros([batch_size, 2]).cpu()
      z[:, 0] = zero_log_prob.cpu()
      z[:, 1] = one_log_prob.cpu()

      preds = torch.argmax(z, dim=1)
      print(zero_log_prob[:10], one_log_prob[:10])
      labels = labels.cpu()
      precision = precision_score(labels[:10], preds[:10])
      recall = recall_score(labels, preds)
      f1 = f1_score(labels, preds)
      print('iter: ' + str(iteration), 'precision : ' + str(precision) + ' recall : ' + str(recall) + ' f1 : ' + str(f1))


### 3-1  AutoEncoder Training

In [30]:
latent_dim=8
model = CNN_sigmaVAE(latent_dim=latent_dim)
model.to(device)
model.cuda() if torch.cuda.is_available() else model.cpu()

model = torch.load('bitch_street_VAEflow.pth')
# model = train_flow_model(model, 1000, .001, train_loader)
test_flow_model(model, validation_loader)

tensor([3.2308, 3.2394, 3.2380, 3.2143, 3.1819, 3.2522, 3.2684, 3.2334, 3.2415,
        3.2404], device='cuda:0', grad_fn=<SliceBackward>) tensor([2.3601, 2.3256, 2.3442, 2.3586, 2.3740, 2.3546, 2.3220, 2.3559, 2.3474,
        2.3377], device='cuda:0', grad_fn=<SliceBackward>)
iter: 0 precision : 0.0 recall : 0.0 f1 : 0.0
tensor([3.2423, 3.2423, 3.2339, 3.2696, 3.2764, 3.2547, 3.2600, 3.2481, 3.2335,
        3.2546], device='cuda:0', grad_fn=<SliceBackward>) tensor([2.3682, 2.3673, 2.4107, 2.3433, 2.3393, 2.3849, 2.3745, 2.3805, 2.3738,
        2.3650], device='cuda:0', grad_fn=<SliceBackward>)
iter: 1 precision : 0.0 recall : 0.0 f1 : 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tensor([3.2682, 3.2510, 3.2577, 3.2663, 3.2425, 3.2574, 3.2514, 3.2594, 3.2623,
        3.2472], device='cuda:0', grad_fn=<SliceBackward>) tensor([2.4127, 2.4038, 2.3971, 2.3778, 2.4163, 2.3837, 2.4054, 2.3687, 2.3843,
        2.3779], device='cuda:0', grad_fn=<SliceBackward>)
iter: 2 precision : 0.0 recall : 0.0 f1 : 0.0
tensor([3.2248, 3.2361, 3.2005, 3.2733, 3.2537, 3.2540, 3.1919, 3.1966, 3.2158,
        3.2406], device='cuda:0', grad_fn=<SliceBackward>) tensor([2.3330, 2.3434, 2.3644, 2.3327, 2.3003, 2.3069, 2.3824, 2.3789, 2.3616,
        2.3052], device='cuda:0', grad_fn=<SliceBackward>)
iter: 3 precision : 0.0 recall : 0.0 f1 : 0.0
tensor([3.2204, 3.2552, 3.2515, 3.2420, 3.2195, 3.2187, 3.2373, 3.2693, 3.2483,
        3.2309], device='cuda:0', grad_fn=<SliceBackward>) tensor([2.4146, 2.3744, 2.3950, 2.3873, 2.4228, 2.4244, 2.3995, 2.3949, 2.3929,
        2.4136], device='cuda:0', grad_fn=<SliceBackward>)
iter: 4 precision : 0.0 recall : 0.0 f1 : 0.0
tensor([3.2119, 3.2046, 3.24

RuntimeError: ignored

## Submission

In [ ]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
opt_th = 0.5

if not train_mode:
    for (test_df, pred_df) in env_iter:
        
        if test_df['weight'].item() > 0:
            test_df = test_df.loc[:, features].values
            if np.isnan(test_df[:, 1:].sum()):
                test_df[:, 1:] = np.nan_to_num(test_df[:, 1:]) + np.isnan(test_df[:, 1:]) * f_mean

            pred_vector = np.mean([fit(model, test_df) for model in models],axis=0)
            pred = np.mean(pred_vector)
            pred_df.action = (pred_vector > opt_th).astype(int) 
            

        else:
            pred_df.action = 0
        env.predict(pred_df)